# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [74]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import math

from dateutil.relativedelta import relativedelta
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

In [75]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("../data/wikipedia_train3.csv", parse_dates=['date'])
test = pd.read_csv("../data/wikipedia_test3.csv", parse_dates=['date'])

In [89]:
train_unmelted = train.set_index(['Page', 'date'])['Visits'].unstack().reset_index()

In [90]:
test_unmelted = test.set_index(['Page', 'date'])['Visits'].unstack().reset_index()

In [91]:
merged_data = pd.merge(train_unmelted, test_unmelted, on=["Page"])

In [92]:
def encode(df):
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    column_name = 'Page'
    if df[column_name].dtype == object:
        df[column_name] = le.fit_transform(df[column_name])
    else:
        pass

In [93]:
encode(merged_data)

In [94]:
processed_train = merged_data.iloc[:, :-62]

In [95]:
processed_test = merged_data.iloc[:, -62:]

In [96]:
processed_test["Page"] = merged_data.iloc[:, 0]

In [97]:
melted_train = pd.melt(processed_train, id_vars=["Page"], value_name="Visits", var_name="date")

In [98]:
melted_test = pd.melt(processed_test, id_vars=["Page"], value_name="Visits", var_name="date")

In [122]:
train.Visits.mean()

12211.298681864537

In [56]:
train_unmelted.head(3)

date,Page,2016-01-01 00:00:00,2016-01-02 00:00:00,2016-01-03 00:00:00,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,2016-01-07 00:00:00,2016-01-08 00:00:00,2016-01-09 00:00:00,...,2016-08-22 00:00:00,2016-08-23 00:00:00,2016-08-24 00:00:00,2016-08-25 00:00:00,2016-08-26 00:00:00,2016-08-27 00:00:00,2016-08-28 00:00:00,2016-08-29 00:00:00,2016-08-30 00:00:00,2016-08-31 00:00:00
0,"""Weird_Al""_Yankovic_en.wikipedia.org_all-acces...",3214.0,2804.0,2884.0,2700.0,2545.0,2644.0,2386.0,2570.0,2858.0,...,3172.0,3069.0,6222.0,4440.0,3300.0,3816.0,3999.0,3913.0,11899.0,4483.0
1,"""Weird_Al""_Yankovic_en.wikipedia.org_desktop_a...",1169.0,1189.0,1267.0,1372.0,1376.0,1541.0,1410.0,1456.0,1181.0,...,1572.0,1573.0,3828.0,2516.0,1738.0,1621.0,1619.0,1988.0,8818.0,2577.0
2,.xxx_en.wikipedia.org_all-access_all-agents,7089.0,7592.0,7436.0,7032.0,6498.0,6887.0,7209.0,7022.0,7194.0,...,9808.0,10503.0,10057.0,9731.0,11170.0,9556.0,9651.0,9567.0,10232.0,7938.0


In [30]:
test_unmelted.head()

date,Page,2016-09-10,2016-09-11,2016-09-12,2016-09-13,2016-09-14,2016-09-15,2016-09-16,2016-09-17,2016-09-18,...,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-11-06,2016-11-07,2016-11-08,2016-11-09,2016-11-10
0,"""Weird_Al""_Yankovic_en.wikipedia.org_all-acces...",3750.0,3301.0,3450.0,3132.0,3460.0,3369.0,2936.0,3245.0,3327.0,...,2919.0,2776.0,3270.0,3488.0,3682.0,7560.0,3592.0,3235.0,2331.0,4515.0
1,"""Weird_Al""_Yankovic_en.wikipedia.org_desktop_a...",1365.0,1318.0,1762.0,1724.0,1794.0,1862.0,1548.0,1408.0,1477.0,...,1720.0,1638.0,2145.0,2201.0,1751.0,3862.0,2164.0,1961.0,1411.0,2056.0
2,.xxx_en.wikipedia.org_all-access_all-agents,8850.0,8776.0,12686.0,10463.0,7342.0,5326.0,5131.0,4638.0,4676.0,...,4788.0,6021.0,9210.0,8702.0,6215.0,6171.0,4138.0,3896.0,5314.0,6107.0
3,.xxx_en.wikipedia.org_mobile-web_all-agents,7784.0,7680.0,10481.0,8609.0,6375.0,4740.0,4501.0,4000.0,4073.0,...,4210.0,5358.0,8352.0,7997.0,5561.0,5588.0,3661.0,3415.0,4709.0,5406.0
4,0_en.wikipedia.org_desktop_all-agents,19392.0,19512.0,19268.0,18984.0,27496.0,29281.0,29473.0,26884.0,28278.0,...,26079.0,27847.0,28623.0,27873.0,29073.0,29231.0,27627.0,27629.0,27407.0,27986.0


**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

Time Split

In [121]:
new_train, holdout = melted_train[melted_train['date']<'2016-06-24'], melted_train[melted_train['date']>='2016-06-24']

In [83]:
new_train, holdout = train[train['date']<'2016-06-01'], train[train['date']>='2016-06-20']

In [4]:
unmelted_new_train = new_train.set_index(['Page', 'date'])['Visits'].unstack().reset_index()

In [405]:
data = unmelted_new_train[[unmelted_new_train.columns[0]] +list(unmelted_new_train.columns[-15:])]
data = data.melt(id_vars=["Page"]).groupby("Page").median().reset_index()

result = pd.merge(holdout,data,on=["Page"])

result.rename(index=str, columns={"Visits_x": "Visits", "value": "pred_Visits"},inplace=True)

In [406]:
print("SMAPE 15-day median, validation: {:.5f}".format(pandas_smape(result)))

SMAPE 15-day median, validation: 49.55305


In [407]:
data = unmelted_new_train[[unmelted_new_train.columns[0]] +list(unmelted_new_train.columns[-15:])]
data = data.melt(id_vars=["Page"]).groupby("Page").median().reset_index()

result = pd.merge(test,data,on=["Page"])

result.rename(index=str, columns={"Visits_x": "Visits", "value": "pred_Visits"},inplace=True)

In [408]:
print("SMAPE 15-day median, test: {:.5f}".format(pandas_smape(result)))

SMAPE 15-day median, test: 50.35650


In [122]:
new_train['Month']     = new_train["date"].dt.month
new_train['Day']       = new_train["date"].dt.day
new_train['DayOfWeek'] = new_train["date"].dt.dayofweek

/usr/local/Cellar/ipython/6.2.1_1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/ipython/6.2.1_1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/Cellar/ipython/6.2.1_1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [123]:
holdout['Month']     = holdout["date"].dt.month
holdout['Day']       = holdout["date"].dt.day
holdout['DayOfWeek'] = holdout["date"].dt.dayofweek

/usr/local/Cellar/ipython/6.2.1_1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/ipython/6.2.1_1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/Cellar/ipython/6.2.1_1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [124]:
melted_test['Month']     = melted_test["date"].dt.month
melted_test['Day']       = melted_test["date"].dt.day
melted_test['DayOfWeek'] = melted_test["date"].dt.dayofweek

In [103]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [104]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [125]:
train_dates = ['2016-01-01', '2016-03-01', '2016-05-01']

In [126]:
### Your code here
def create_validation(df, start_date):
    return df.loc[(df['date'] >= pd.to_datetime(start_date) - relativedelta(days=0)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=1))].index, \
           df.loc[(df['date'] >= pd.to_datetime(start_date) + relativedelta(months=1)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=2))].index

In [127]:
new_train.shape

(2067625, 6)

In [128]:
### Your validation iterator
CVIterator = []
for i in train_dates:
    trainIndices, valIndices = create_validation(new_train, i)
    CVIterator.append( (trainIndices, valIndices) )

In [129]:
for x,y in CVIterator:
    print (min(x), min(y))

0 366265
708900 1075165
1429615 1795880


In [130]:
X_train = new_train.drop(['date', 'Visits'], axis=1)
y_train = new_train['Visits']

X_test = holdout.drop(['date', 'Visits'], axis=1)
y_test = holdout['Visits']

In [131]:
X = melted_test.drop(['date', 'Visits'], axis=1)
y = melted_test['Visits']

In [132]:
X_test.head(3)

,Page,Month,Day,DayOfWeek
2067625,0,6,24,4
2067626,1,6,24,4
2067627,2,6,24,4


In [133]:
regressor = SGDRegressor()

param_grid = {'regressor__alpha': [0.001, 0.000001], 'regressor__l1_ratio':
              [0.15, 0.35, 0.65], 'regressor__eta0': [0.01, 0.0001]}

pipeline = Pipeline([("regressor", regressor)])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=CVIterator)

%time grid_search.fit(X_train, y_train)

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.6/site-packages/skl

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.6/site-packages/skl

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


CPU times: user 11.1 s, sys: 757 ms, total: 11.9 s
Wall time: 11.9 s


GridSearchCV(cv=[(Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            366255, 366256, 366257, 366258, 366259, 366260, 366261, 366262,
            366263, 366264],
           dtype='int64', length=366265), Int64Index([366265, 366266, ...2067620, 2067621,
            2067622, 2067623, 2067624],
           dtype='int64', length=271745))],
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('regressor', SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'regressor__alpha': [0.001, 1e-06], 'regressor__l1_ratio': [0.15, 0.35, 0.65], 'regressor__eta0': [0.01, 0.0001]},
 

In [134]:
grid_search.best_params_

{'regressor__alpha': 1e-06,
 'regressor__eta0': 0.0001,
 'regressor__l1_ratio': 0.35}

In [135]:
y_pred = grid_search.predict(X_test)

In [136]:
print("SMAPE score for validation set: {:.4f}".format(smape_fast(y_test, y_pred)))

SMAPE score for validation set: 199.5111


In [137]:
print("SMAPE score for test set: {:.4f}".format(smape_fast(y, y_pred)))

SMAPE score for test set: 199.3379


## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [17]:
train = pd.read_csv('../data/train_french.csv')
test = pd.read_csv('../data/test_french.csv')

In [18]:
train['is_train'] = True
test['is_train'] = False

In [19]:
df = pd.concat([train, test], axis=0)

In [20]:
categorical_cols = ['nature_lri', 'niv_urbanisation', 'profession', 'sit_fam']
bin_cols = ['formule', 'objets_precieux', 'pres_contrat_auto', 'societe', 'var11', 'var3', 'var6', 'var7',
           'var9','veranda']
cat_cols = categorical_cols+bin_cols

In [21]:
from collections import defaultdict

def __encode_categorical(df_list, cat_cols):

    d = defaultdict(LabelEncoder)

    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),

    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))

__encode_categorical(df_list=[df], cat_cols=cat_cols)

columns_to_use = bin_cols + categorical_cols

In [7]:
pp.ProfileReport(train)

Number of variables,30
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.4 MiB
Average record size in memory,233.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,1


In [115]:
pp.ProfileReport(test)

Number of variables,29
Number of observations,75968
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [22]:
df.shape

(275968, 30)

In [23]:
strong_features = ['biens_nomades', 'formule', 'patrimoine_mob', 'var11', 'nb_pieces', 
                   'var1', 'var5', 'prime_tot_ttc']

In [24]:
processed_train = df.iloc[:200000, :29]
processed_test = df.iloc[200000:, :29]

In [83]:
def remove_low_correlation_features(df, strong_features):
    for i in df.columns:
        if i not in strong_features:
            df.drop(i, axis=1, inplace=True)

In [84]:
remove_low_correlation_features(processed_test, strong_features=strong_features)

In [88]:
processed_test.head(3)

,nb_pieces,biens_nomades,formule,patrimoine_mob,var1,var5,var11,prime_tot_ttc
0,5,500,1,10731,1,41,0,107.94
1,2,0,1,0,5,100,1,80.41
2,4,1000,1,74572,6,100,1,142.12


In [30]:
processed_train.head(3)

,nb_pieces,biens_nomades,formule,patrimoine_mob,var1,var5,var11,prime_tot_ttc
0,1,500,0,13341,4,49,1,67.68
1,2,500,1,11944,3,100,0,120.60
2,3,500,1,5923,2,100,1,100.62


In [27]:
from sklearn.model_selection import train_test_split

Random split

In [28]:
new_train, validation = train_test_split(processed_train, test_size = 0.28)

In [76]:
X_train, y_train = new_train.drop("prime_tot_ttc", axis=1), new_train['prime_tot_ttc']

X_test, y_test = validation.drop("prime_tot_ttc", axis=1), validation['prime_tot_ttc']

In [92]:
X, y = processed_test.drop("prime_tot_ttc", axis=1), processed_test['prime_tot_ttc']

In [34]:
from sklearn.ensemble import RandomForestRegressor

reg = DecisionTreeRegressor()

pipeline = Pipeline([("regressor", reg)])

param_grid = {"regressor__criterion": ["mae"], 
              "regressor__max_depth": [4], 
              "regressor__min_samples_leaf": [5],
             "regressor__min_samples_split": [10]}

grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=3)

%time grid_search.fit(X_train, y_train)

CPU times: user 13min 36s, sys: 1.4 s, total: 13min 37s
Wall time: 27min 7s


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('regressor', DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'regressor__criterion': ['mae'], 'regressor__max_depth': [4], 'regressor__min_samples_leaf': [5], 'regressor__min_samples_split': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [35]:
from sklearn.metrics import mean_absolute_error

In [103]:
y_pred = grid_search.predict(X_test)

In [104]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [105]:
print("MAPE validation: {:.4f}%".format(mean_absolute_percentage_error(y_test, y_pred)))

MAPE validation: 17.3539%


In [106]:
y_pred = grid_search.predict(X)

In [107]:
print("MAPE test: {:.4f}%".format(mean_absolute_percentage_error(y, y_pred)))

MAPE test: 17.3356%


In [111]:
train.corr()

,code_postal,nb_pieces,biens_nomades,societe,domaine_activite,age_societaire,objets_precieux,pres_contrat_auto,formule,sit_juridique,...,var3,var4,var5,var6,var7,var8,var10,var11,prime_tot_ttc,is_train
code_postal,1.000000,-0.001201,0.002187,-0.005337,-0.001730,0.001500,0.003194,0.004218,0.004006,0.000766,...,0.001152,-0.003255,0.004071,0.003432,-0.000350,-0.000831,0.000041,-0.002047,-0.016662,NaN
nb_pieces,-0.001201,1.000000,0.353435,-0.136759,-0.131434,0.161995,0.143226,0.160223,0.019950,0.177838,...,0.051187,0.011366,-0.073852,0.073123,0.085940,-0.232176,-0.037349,-0.001476,0.290765,NaN
biens_nomades,0.002187,0.353435,1.000000,-0.082532,-0.074477,0.152273,0.305959,0.060236,0.095988,-0.171434,...,0.006459,0.005397,-0.214433,0.073377,0.100551,-0.121427,-0.042762,-0.286005,0.483270,NaN
societe,-0.005337,-0.136759,-0.082532,1.000000,0.581906,-0.302584,-0.054727,-0.257580,-0.025014,-0.201504,...,-0.292761,-0.002535,-0.062171,-0.154325,-0.015080,0.231601,0.161539,0.036770,-0.092452,NaN
domaine_activite,-0.001730,-0.131434,-0.074477,0.581906,1.000000,-0.214576,-0.047752,-0.246166,-0.029467,-0.215219,...,-0.258368,0.000263,-0.057753,-0.131449,-0.022290,0.261314,0.168609,0.047062,-0.093819,NaN
age_societaire,0.001500,0.161995,0.152273,-0.302584,-0.214576,1.000000,0.169058,0.111931,0.010446,0.389994,...,-0.090805,0.008289,0.107904,0.208603,0.016925,-0.405528,-0.357364,-0.056920,0.143474,NaN
objets_precieux,0.003194,0.143226,0.305959,-0.054727,-0.047752,0.169058,1.000000,0.017334,-0.003284,0.035884,...,-0.031827,0.007319,-0.052105,0.067365,0.050309,-0.074349,-0.088151,-0.089697,0.177694,NaN
pres_contrat_auto,0.004218,0.160223,0.060236,-0.257580,-0.246166,0.111931,0.017334,1.000000,0.011883,0.150851,...,0.116345,0.002614,0.050515,0.172447,0.012319,-0.197958,-0.195818,-0.029634,0.075166,NaN
formule,0.004006,0.019950,0.095988,-0.025014,-0.029467,0.010446,-0.003284,0.011883,1.000000,0.003641,...,0.017444,0.001864,-0.010296,0.005218,0.007040,-0.033902,0.006376,-0.039753,0.519445,NaN
sit_juridique,0.000766,0.177838,-0.171434,-0.201504,-0.215219,0.389994,0.035884,0.150851,0.003641,1.000000,...,0.018701,0.006390,0.186693,0.152641,0.027948,-0.358287,-0.194409,0.028581,0.042705,NaN
